In [ ]:
Bayesian Optimization 

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Set the number of Mel-Frequency Cepstral Coefficients (MFCCs) to extract.
num_mfcc = 20

# Initialize empty arrays to hold the feature matrix from audio and corresponding labels.
X_train = []
y_train = []

# Iterate through all of the .wav files in the training directory.
for file in sorted(os.listdir(data_path)):
    if file.endswith(".wav"):
        # Load the audio file using librosa and our files in Google Drive.
        audio_data, sample_rate = librosa.load(os.path.join(data_path, file))

        # Determine the MFCC features from the audio data provided above.
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=num_mfcc)


        # Grab the mean and standard deviation of our features.
        mfccs_avg = np.mean(mfccs, axis=1)
        mfccs_std = np.std(mfccs, axis=1)

        # Add the MFCC features to the data matrix
        X_train.append(np.concatenate((mfccs_avg, mfccs_std)))

        # Determine the emotion label of the audio file based on its filename.
        label = ""
        idx = 0
        while not (file[idx].isdigit()):
            label += file[idx]
            idx += 1
        y_train.append(label)




# Define the hyperparameter search space
params = {
    "C": Real(1e-6, 1e+6, prior="log-uniform"),
    "gamma": Real(1e-6, 1e+1, prior="log-uniform"),
    "kernel": Categorical(["poly", "sigmoid"]),
    "degree": Integer(1, 5),
    "coef0": Real(-1, 1)
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Define the Skopt hyperparameter search
skopt_search = BayesSearchCV(
    svm_classifier,
    params,
    n_iter=20,
    cv=5,
    n_jobs=-1,
    scoring="accuracy",
    random_state=42
)

# Apply feature scaling to the dataset
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

# Perform the hyperparameter search with Skopt
#skopt_search.fit(X_train, y_train)
skopt_search.fit(X_train_scaled, y_train)

# Print the best hyperparameters and score found by Skopt
print("Best parameters:", skopt_search.best_params_)
print("Best score:", skopt_search.best_score_)

Data processing

In [ ]:
# Initialize the SVM classifier with the best hyperparameters found by Skopt
svm_classifier = SVC(C=skopt_search.best_params_['C'], gamma=skopt_search.best_params_['gamma'], kernel=skopt_search.best_params_['kernel'])

# Use stratified sampling to split the data into a smaller training set and a validation set
X_train_new, X_val, y_train_new, y_val = train_test_split(
     X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

svm_classifier.fit(X_train_new, y_train_new)


# Predict the labels of the validation set using the trained SVM classifier
y_pred_val = svm_classifier.predict(X_val)

# Predict the labels of the validation set using the trained SVM classifier
y_pred_val = svm_classifier.predict(X_val)

# Print the classification report and confusion matrix for the validation set
print(classification_report(y_val, y_pred_val))
print(confusion_matrix(y_val, y_pred_val))

# Predict the labels of the test data using the trained SVM classifier
X_test = []
file_names = []
for file in sorted(os.listdir(test_path)):
    if file.endswith(".wav"):

        file_names.append(file.split(".")[0])

        # Load the audio file using librosa and our files in Google Drive.
        audio_data, sample_rate = librosa.load(os.path.join(test_path, file))

        # Determine the MFCC features from the audio data provided above.
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=num_mfcc)

        # Grab the mean and standard deviation of our features.
        mfccs_avg = np.mean(mfccs, axis=1)
        mfccs_std = np.std(mfccs, axis=1)

        # Add the MFCC features to the data matrix
        X_test.append(np.concatenate((mfccs_avg.reshape(-1), mfccs_std.reshape(-1))))

X_test = np.array(X_test)

# Predict the labels of the test data using the trained SVM classifier
y_pred_test = svm_classifier.predict(X_test)

# Convert the file names and predicted labels to a two-dimensional numpy array
predictions = np.column_stack((file_names, y_pred_test))

# Sort the predictions array by the first column (the sorted file names)
predictions = predictions[np.lexsort((predictions[:, 0],))]

# Save the predictions to a CSV file
np.savetxt("predictions.csv", predictions, fmt='%s', delimiter=",", encoding='utf-8')
with open("predictions.csv", "r+") as f:
    content = f.read()
    f.seek(0, 0)
    f.write("filename,Label\n" + content)